# 載入套件

In [40]:
import cv2
import numpy as np
import gradio as gr

# CSS樣式

In [41]:
# --- CSS 樣式 ---
# 負責隱藏多餘圖示、設定滑桿兩端標籤文字、優化容器寬度
CUSTOM_CSS = """
/* 1. 隱藏頁尾與圖片框內部的預設標記 (如: Drop Image Here) */
footer, .label-wrap { display: none !important; }

/* 2. 精確隱藏圖片組件上的功能按鈕 (編輯、複製、Webcam) */
button[aria-label="Edit"], 
button[aria-label="Copy"], 
button[aria-label="Capture from device"] { 
    display: none !important; 
}

/* 3. 自定義滑桿容器：添加 Smooth/Detailed 文字標籤 */
#slider-container { 
    position: relative; 
    padding-bottom: 25px; 
    width: 100% !important; 
}

#slider-container::before { 
    content: "Smooth"; 
    position: absolute; left: 0; bottom: 5px; 
    font-size: 0.9em; color: #666; font-weight: bold; 
}

#slider-container::after { 
    content: "Detailed"; 
    position: absolute; right: 0; bottom: 5px; 
    font-size: 0.9em; color: #666; font-weight: bold; 
}

/* 4. 設定整體介面寬度 */
.gradio-container { max-width: 1100px !important; }
"""

# 卡通化運算

In [42]:
def cartoonize_image_logic(img, intensity):
    # 異常處理：若輸入為空則不執行
    if img is None: return None
    
    # 1. 穩定化縮圖：防止高解析度圖片造成傳輸協議錯誤 (h11 Content-Length 錯誤)
    target_size = 800 
    h, w = img.shape[:2]
    if max(h, w) > target_size:
        scale = target_size / max(h, w)
        img = cv2.resize(img, (int(w * scale), int(h * scale)))

    # 2. 參數映射：將滑桿 0-100 的值轉換為濾鏡參數
    filter_iterations = int(intensity / 10) + 1  # 雙邊濾波次數
    sigmaColor = int(intensity * 2) + 10         # 顏色融合強度
    base_block = int(intensity / 5)              # 輪廓線粗細基準
    
    # OpenCV 的 adaptiveThreshold 要求 blockSize 必須為 >1 的奇數
    blockSize = base_block if base_block % 2 == 1 else base_block + 1
    if blockSize < 3: blockSize = 3
    
    # 3. 影像卡通化流程
    img_bgr = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    
    # A. 雙邊濾波 (Bilateral Filter)：平滑色彩並保留邊緣
    # 先縮小再模糊，可以強化塊狀感並加快速度
    img_small = cv2.pyrDown(cv2.pyrDown(img_bgr))
    for _ in range(filter_iterations):
        img_small = cv2.bilateralFilter(img_small, d=9, sigmaColor=sigmaColor, sigmaSpace=7)
    
    # 放大回原尺寸
    img_cartoon = cv2.pyrUp(cv2.pyrUp(img_small))
    img_cartoon = cv2.resize(img_cartoon, (img_bgr.shape[1], img_bgr.shape[0]))
    
    # B. 邊緣偵測 (Edge Detection)：提取黑色輪廓線
    img_gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
    img_blur = cv2.medianBlur(img_gray, 7) # 減少雜訊
    img_edge = cv2.adaptiveThreshold(
        img_blur, 255, 
        cv2.ADAPTIVE_THRESH_MEAN_C, 
        cv2.THRESH_BINARY, 
        blockSize, 2
    )
    
    # C. 合併結果：將彩色塊與黑色輪廓合併
    img_edge_colored = cv2.cvtColor(img_edge, cv2.COLOR_GRAY2BGR)
    cartoon_bgr = cv2.bitwise_and(img_cartoon, img_edge_colored)
    
    # 將 BGR 轉回 RGB 供介面顯示
    return cv2.cvtColor(cartoon_bgr, cv2.COLOR_BGR2RGB)

# Gradio GUI 介面設計

In [43]:
with gr.Blocks(css=CUSTOM_CSS) as demo:
    gr.Markdown("## 🎨 影像卡通化轉換器")
    
    with gr.Row():
        # 左側：原始影像 (僅開啟上傳功能，並允許互動以顯示清除按鈕)
        input_img = gr.Image(label="原始影像", type="numpy", sources=["upload"], interactive=True)
        # 右側：輸出結果 (設定為不可手動上傳)
        output_img = gr.Image(label="卡通化結果", interactive=False)
    
    # 橫跨全寬的滑桿區
    with gr.Column(elem_id="slider-container"):
        intensity_slider = gr.Slider(
            minimum=0, 
            maximum=100, 
            value=50, 
            step=1, 
            label="風格調整"
        )

    # 事件綁定
    # 當使用者放開滑桿時執行 (避免過度計算)
    intensity_slider.release(cartoonize_image_logic, [input_img, intensity_slider], output_img)
    # 當使用者更換圖片時立即執行
    input_img.change(cartoonize_image_logic, [input_img, intensity_slider], output_img)

# 執行程式

In [44]:
# 1. 關閉之前運行的舊實例以釋放端口
demo.close()
# 2. 啟動介面
demo.launch()

Running on local URL:  http://127.0.0.1:7900

To create a public link, set `share=True` in `launch()`.
